### Capstone Project: NYC-311 Service Requests Analysis by Heriberto Varela.

In [141]:
import time
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import squarify
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import re
from wordcloud import WordCloud

### Modeling

In [237]:
# dataset with columns added after EDA
req_df = pd.read_parquet("req_df_for_model.parquet.gzip")

The model that will be trained with the request data will be a regression model. The purpose of this model is to predict the resolution time it will take for a request to be resolved. 

The target variable will be time until resolution in seconds. This format was chosen given that a considerable amount of requests were marked completed in less than a day.

In [238]:
# timedelta in seconds
req_df['seconds_until_resolution'] = req_df['time_until_resolution'].dt.total_seconds()

Modeling will be started with a set of baseline columns, including demographic data, a binary variable representing the complaint grouping created in the EDA process explained before.

In [239]:
baseline_cols = ['agency', 'borough', 'community_board', 'median_income_2022', 'poverty_rate_2021','diversity_index_2021', \
                 'street_complaints','noise_complaints', 'building_complaints', 'cleanliness_complaints', 'safety_complaints', \
                 'seconds_until_resolution', 'days_until_resolution', 'descriptor']

In [240]:
model_df = req_df[baseline_cols].loc[req_df['closed']==1]
model_df.dropna(inplace=True)
model_df.reset_index(drop=True, inplace=True)

In [241]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619874 entries, 0 to 2619873
Data columns (total 14 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   agency                    string 
 1   borough                   string 
 2   community_board           string 
 3   median_income_2022        float64
 4   poverty_rate_2021         float64
 5   diversity_index_2021      float64
 6   street_complaints         int64  
 7   noise_complaints          int64  
 8   building_complaints       int64  
 9   cleanliness_complaints    int64  
 10  safety_complaints         int64  
 11  seconds_until_resolution  float64
 12  days_until_resolution     float64
 13  descriptor                string 
dtypes: float64(5), int64(5), string(4)
memory usage: 279.8 MB


We'll start our model with only the demographic data.

In [242]:
X = model_df[['median_income_2022','poverty_rate_2021','diversity_index_2021']]
y = model_df['seconds_until_resolution']

Using a linear regression we can get a baseline score, using only the demographic data to make predictions.

In [245]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression() 

lr.fit(X_train, y_train)

LinearRegression()

In [246]:
lr.score(X_test, y_test)

0.0012706927443059124

Our score in this case is R2 (r-squared), which shows how well the model predicts the outcome of the target variable. The value returned in our baseline model means that the model explains or predicts close to 0% of the relationship between the target variable and our other variables.

On the second model trained, we'll use demographic data with the complaint grouping classification for each request.

In [247]:
X_base = model_df[['median_income_2022','poverty_rate_2021','diversity_index_2021', 'street_complaints','noise_complaints', \
              'building_complaints', 'cleanliness_complaints', 'safety_complaints']]
y = model_df['seconds_until_resolution']

In [248]:
X_train, X_test, y_train, y_test = train_test_split(X_base,y,test_size = .20)

In [249]:
lr = LinearRegression() 

lr.fit(X_train, y_train)

LinearRegression()

In [250]:
lr.score(X_test, y_test)

0.05092625487003388

Our score goes up to 5%, which means 5% of the relationship is explained by the demographic data and the complaint grouping classification.

To further include classification information from the requests, we'll use a one-hot encoding of the agencies tasked with their resolution, and the borough where the request originated. 

In [251]:
from sklearn.preprocessing import OneHotEncoder

# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
category = pd.DataFrame(model_df['agency'])
agency_encoded = ohe.fit_transform(category)
dense_array = agency_encoded.toarray()
# Put into a dataframe to get column names
agency_encoded_df = pd.DataFrame(dense_array, columns=ohe.categories_[0], dtype=int)
agency_encoded_df.head()

,DCWP,DEP,DHS,DOB,DOE,DOHMH,DOT,DPR,DSNY,EDC,HPD,NYC311-PRD,NYPD,OTI,TLC
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [252]:
X_agency = pd.concat([X_base, agency_encoded_df], axis=1)
y = model_df['seconds_until_resolution']

In [253]:
X_train, X_test, y_train, y_test = train_test_split(X_agency,y,test_size = .20)

In [254]:
lr = LinearRegression() 

lr.fit(X_train, y_train)

LinearRegression()

In [255]:
lr.score(X_test, y_test)

0.21910613446136873

In [256]:
category = pd.DataFrame(model_df['borough'])
borough_encoded = ohe.fit_transform(category)
dense_array = borough_encoded.toarray()
# Put into a dataframe to get column names
borough_encoded_df = pd.DataFrame(dense_array, columns=ohe.categories_[0], dtype=int)
borough_encoded_df.head()

,BRONX,BROOKLYN,MANHATTAN,QUEENS,STATEN ISLAND
0,1,0,0,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,1,0,0,0


In [257]:
X_boroughs = pd.concat([X_base, agency_encoded_df, borough_encoded_df], axis=1)
y = model_df['seconds_until_resolution']

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X_boroughs,y,test_size = .20)

In [259]:
lr = LinearRegression() 

lr.fit(X_train, y_train)

LinearRegression()

In [260]:
lr.score(X_test, y_test)

0.21916444127108836

Including this classification information brings our R2 score up to 21%. This is a significant improvement from our previous model.

For the next model, we'll include the community board classification instead of the borough classification of each request. This is to see if including a more specific geographical classification yields better predictions.

In [261]:
category = pd.DataFrame(model_df['community_board'])
community_board_encoded = ohe.fit_transform(category)
dense_array = community_board_encoded.toarray()
# Put into a dataframe to get column names
community_board_encoded_df = pd.DataFrame(dense_array, columns=ohe.categories_[0], dtype=int)
# Show
community_board_encoded_df.head()

,01 BRONX,01 BROOKLYN,01 QUEENS,01 STATEN ISLAND,02 BRONX,02 BROOKLYN,02 MANHATTAN,02 QUEENS,02 STATEN ISLAND,03 BROOKLYN,...,12 MANHATTAN,12 QUEENS,13 BROOKLYN,13 QUEENS,14 BROOKLYN,14 QUEENS,15 BROOKLYN,16 BROOKLYN,17 BROOKLYN,18 BROOKLYN
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
X_boards = pd.concat([X_base, community_board_encoded_df, agency_encoded_df], axis=1)
y = model_df['seconds_until_resolution']

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X_boards,y,test_size = .20)

In [264]:
lr = LinearRegression() 

lr.fit(X_train, y_train)

LinearRegression()

In [265]:
lr.score(X_test, y_test)

0.22084155515815362

The score went up by >1% compared to the last model trained with borough classification.

---

Now we'll try training other regression models.

In [270]:
from sklearn.tree import DecisionTreeRegressor

In [271]:
X_train, X_test, y_train, y_test = train_test_split(X_boroughs,y,test_size = .20)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [272]:
dt = DecisionTreeRegressor()

dt.fit(X_train,y_train)

DecisionTreeRegressor()

In [273]:
dt.score(X_test, y_test)

0.2645395517177195

The DecisionTreeRegressor returned a score of 26%, a ~5% increase from our linear regression model. This could be because of the decision trees' handling of outlier data.

In [274]:
from xgboost.sklearn import XGBRegressor

In [275]:
X_train, X_test, y_train, y_test = train_test_split(X_boroughs,y,test_size = .20)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [131]:
xgb = XGBRegressor()

xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [132]:
xgb.score(X_test, y_test)

0.2685802374662174

Next we try using the XGBRegressor, which gave a score of around 26% as well. This model was chosen in an effort to boost our model's performance.

---

As a final effort to improve our regression model, we'll use scaling and a grid search to find the best hyperparameters for our XGBRegressor implementation.

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X_boroughs,y,test_size = .20)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [134]:
from sklearn.pipeline import Pipeline

estimators = [
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
]

my_pipe = Pipeline(estimators)

In [135]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'scaler': [StandardScaler()],
        'model': [XGBRegressor()],
        'model__max_depth': [5, 10],
        'model__learning_rate': [0.7, 1.0],
        'model__min_child_weight': [5, 10, 15],
        'model__gamma': [0.2]
    }
]

grid = GridSearchCV(my_pipe, param_grid, cv=5)

In [136]:
fittedgrid = grid.fit(X_train, y_train)

In [137]:
fittedgrid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0.2, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.7,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None, min_child_weight=10,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [138]:
fittedgrid.best_params_

{'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.7, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
 'model__gamma': 0.2,
 'model__learning_rate': 0.7,
 'model__max_depth': 5,
 'model__min_child_weight': 10,
 'scaler': StandardScaler()}

In [139]:
fittedgrid.score(X_test, y_test)

0.2655683801354636

After using GridSearch to optimize the hyperparameters of XGBRegressor, we didn't see a significant increase in the R2 score. Our best performing model explains around 26% of the variance in our data. Further improvements can be made by including more relevant features like additional demographic data, a better encoding for the categorical classifications, and a more balanced distribution of our target variable, which in this case was the time until a request was resolved in seconds.